# 特征

2021年9月16日，漱玉平民出现低位高换手，换手率18.4%，上涨4.4%，上影线5.6%。日线量比达4.8，30分钟量比更是高达29.5倍。其后不久，连续出现20cm涨停。

又如上声电子， 2021-11-03前后出现上述特征，随后一周上涨50%以上。

需要直接使用jqdatasdk

买入点特征：
1. 新股或者次新股（<1年）
2. 换手率15%以上
3. 连续两日以上涨幅5%以上
4. 低位: 120日altitude < 0.3
5. RSI？

In [1]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsiday, rsi30
import datetime
import os

await init_notebook()

# 高换手版（适用于新股？）

In [ ]:
def features(code, name, bars, results, frame_type):
    altitude_threshold =  0.6
    lb_threshold =  3
    roc_threshold = 0.05
    
    high = bars["high"]
    close = bars["close"]
    low = bars["low"]
    open_ = bars["open"]

    volume = bars["volume"]

    c2, c1, c0 = close[-3:]
    v2, v1, v0 = volume[-3:]
    
    lv = np.min(volume[-6:-1])
    
    hh = np.max(high)
    ll = np.min(low)
    
    altitude = 1 - (hh - close[-1])/(hh - ll)
    
    roc2, roc1, roc0 = close[-3:] / close[-4:-1] - 1
    
    to2 = jq_get_turnover_realtime(code, v2, c2)
    to1 = jq_get_turnover_realtime(code, v1, c1)
    to0 = jq_get_turnover_realtime(code, v0, c0)
    
    if len(days) < 60:
        return
    
    results.append([name, code, altitude, roc2, roc1, roc0, to2, to1, to0, days])

results = await scan(features, 60, nstocks=-1, silent=True)

In [ ]:
results

# 高量比版

参考标的：翰宇药业， 2021年11月9日前后

1. 量比极高
2. 平台突破

In [ ]:
def features(code, name, bars, results, frame_type):
    close = bars["close"]
    volume = bars["volume"]
    high = bars["high"]
    low = bars["low"]
    
    v1, v0 = top_volume_direction(bars)
    vn = moving_net_volume(bars)
    
#     mas = []
#     for win in [5, 10, 20, 30, 60]:
#         mas.append(moving_average(close, win)[-20:])
        
#     mas_max = np.max(mas, axis=0)
#     mas_min = np.min(mas, axis=0)
#     convergency = 1 - mas_min/mas_max
    
#     # count of convergency
#     coc = np.count_nonzero(convergency < 0.1)/20
    alt = altitude(bars)
    
    results.append((name, code, alt, v1, v0, vn[-1]))

In [ ]:
results = await scan(features, 120, '1d', nstocks=-1, silent=True)
df = pd.DataFrame(results, columns=["name", "code", "alt", "v1", "v0", "vn"])

In [ ]:
df[(df.v1>2.5)&(df.v0>-0.5)&(df.vn>10)&(df.alt<0.6)&(df.v0<0)]

# V1 低位放量

In [ ]:
async def backtest_v1(start, end):
    start = arrow.get(start).date()
    end = arrow.get(end).date()
    
    signals = []
    for code in Stock.choose(["stock"]):
        sec = Stock(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)

            for i in range(120, len(bars) - 15):
                xbars = bars[i-120:i]

                signal = fire_long(code, xbars, pcr=0.2)
                if signal is None:
                    continue
                else:
                    frame, price, altitude, lb, to, pcr_0 = signal
                    row = [sec.display_name, code, frame, price, altitude, lb, to, pcr_0]
                    for j in [1, 3, 5, 10, 15]:
                        ybars = bars[i:i+j]
                        yclose = np.max(ybars["close"])
                        row.append(yclose/price-1)

                    signals.append(row)
        except Exception:
            pass
        
    columns = ["name", "code", "frame", "price", "altitude", "lb", "to", "pcr_0", 
               "pcr_1", "pcr_3", "pcr_5", "pcr_10", "pcr_15"]
    return pd.DataFrame(signals, columns=columns)
        
df = await backtest_v1("2021-01-04", "2021-09-30")
df

In [ ]:
df.describe()

# V2 加上次新股属性

回测表明，次新股低位放量后，10交易日平均涨幅为12%；15日平均涨幅为15%。

In [ ]:
async def backtest_v2(start, end, **params):
    end = arrow.get(end).date()
    start = arrow.get(start).date()
    
    signals = []
    for code in Stock.choose(["stock"]):
        sec = Stock(code)
        
        if tf.day_shift(sec.ipo_date, 60) < start:
            continue
            
        try:
            bars = await sec.load_bars(sec.ipo_date, end, FrameType.DAY)

            for i in range(60, min(250, len(bars) - 15)): #限制在一年内
                xbars = bars[i-60:i]

                signal = fire_long(code, xbars, **params)
                if signal is None:
                    continue
                else:
                    frame, price, altitude, lb, to, pcr_0 = signal
                    row = [sec.display_name, code, frame, price, altitude, lb, to, pcr_0]
                    for j in [1, 3, 5, 10, 15]:
                        ybars = bars[i:i+j]
                        yclose = np.max(ybars["close"])
                        row.append(yclose/price-1)

                    signals.append(row)
        except Exception:
            pass
        
    columns = ["name", "code", "frame", "price", "altitude", "lb", "to", "pcr_0", 
               "pcr_1", "pcr_3", "pcr_5", "pcr_10", "pcr_15"]
    return pd.DataFrame(signals, columns=columns)

In [ ]:
df = await backtest_v2("2015-01-04", "2021-09-30", altitude=0.6, turnover=0.07, lb=3, pcr=0.05)
df.describe()

In [ ]:
report("低位放量v2次新股版回测结果", {"altitude":0.6, "turnover":0.07, "lb":3, "pcr":0.05}, result_df=df)

# v3
放量后，缩量下跌至低于放量涨当日开盘价买入，计算持有1，3，5，10，15日，以收盘价卖出的利润。结果表明并无改进。

In [ ]:
async def backtest_v3(start, end):
    start = arrow.get(start).date()
    end = arrow.get(end).date()
    
    signals = []
    
    # 寻找发出放量信号的个股
    for code in Stock.choose(["stock"]):
        sec = Stock(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)

            for i in range(120, len(bars) - 30):
                xbars = bars[i-120:i]

                signal = fire_long(code, xbars, pcr=0.3, altitude=0.8, lb=2.5)
                if signal is None:
                    continue
                else:
                    frame, close, altitude, lb, to, pcr_0 = signal
                    row = [code, frame, altitude, lb, to]
                    signals.append(row)
        except Exception:
            pass
        
    # 如果股价回到放量日开盘价以下，则为买入点
    results = []
    for code, sig_start, altitude, lb, to in signals:
        sig_end = tf.day_shift(sig_start, 15)
        sec = Stock(code)
        try:
            bars = await sec.load_bars(sig_start, sig_end, FrameType.DAY)
            open_price = bars["open"][0]
            
            sig_bars = bars[1:]
            pos = np.argwhere(sig_bars["low"] <= open_price).flatten()
            if len(pos) == 0:
                continue
                
            order_day = sig_bars[pos[0]]["frame"]
            ystart = tf.day_shift(order_day, 1)
            yend = tf.day_shift(order_day, 16)
            
            ybars = await sec.load_bars(ystart, yend, FrameType.DAY)
            
            row = [sec.display_name, code, sig_start, order_day, open_price, altitude, lb, to]
            for n in [1, 3, 5, 10, 15]:
                c = ybars[n]["close"] / open_price - 1
                row.append(c)
            
            results.append(row)
        except Exception as e:
            break
        
    columns = ["name", "code", "fire_date", "oder_date", "order_price", "altitude", "lb", "to",  
               "pcr_1", "pcr_3", "pcr_5", "pcr_10", "pcr_15"]
    return pd.DataFrame(results, columns=columns)
        
df = await backtest_v3("2021-01-04", "2021-09-30")
df

# V4
找出近10天日线放量最大超过4倍，或者换手率超过20%的标的，30分钟线出现rsi低于25%时发出提示。

In [ ]:
def daybars_trigger(name, code, bars, results, frame_type):
    n = 10
    last_n = bars[-n:]
    volume = last_n["volume"]
    vr = max(volume)/min(volume)
#     if vr < 4:
#         return
    
    pos = np.argmax(volume)
    flag = last_n[pos]["close"] < last_n[pos]["open"] #阴线放量
    
    results.append([name, code, vr, n-pos, flag])
    
result = await scan(daybars_trigger, 15, tm="2021-11-05")

In [ ]:
df = pd.DataFrame(result, columns=["code", "name", "vr", "dist", "flag"])
pd.options.display.max_rows=999
df.nlargest(10, "vr")

# V5 周线版

2021年11月05那周，合力科技（603917）周线放量拉升24%。此后一周持续

In [ ]:
def features(name, code, bars, results, frame_type):
    if len(bars) < 10:
        return
    
    volume = bars["volume"][-10:]
    v1, v0 = np.sum(volume[:5]), np.sum(volume[5:])
    
    vr = v0/v1
    
    row = (name, code, vr)
    results.append(row)
    
results = await scan(trigger, 5, frame_type = '1w', tm = '2021-10-15 15:00', nstocks=-1)
df = pd.DataFrame(features, columns=["name", "code", "vr"])

In [ ]:
df = df.nlargest(20, "vr")

In [ ]:
async def evaluate(df):
    results = []
    for code, name, vr in df.to_numpy():
        bars = await get_bars(code, 22, end="2021-11-16 15:00")
        results.append((code, name, vr, *performance(bars)))
        
    columns = ["code", "name", "vr", "max", "min", "1", "3", "5", "8", "13", "21"]
    return pd.DataFrame(results, columns=columns)

In [ ]:
perf = await evaluate(df)

# 30分钟版

寻找10：00放量，20日内涨幅不大的个股，在14：30分钟，如果保持上升（大于10：00收盘），则发出信号

In [3]:
async def pooling(tm=None, codes=None):
    """找出10:00时放量的个股前100名"""
    day = arrow.get(tm).date() if tm else arrow.now().date()
    end = datetime.datetime(day.year, day.month, day.day, 10)
    
    def find_vr(code, name, bars, results, ft):
        # 早盘10点，30分钟放量
        volume = bars["volume"]
        
        c = bars["close"]
        o = bars["open"]
        h = bars["high"]
        
        # 收阳且上涨, **冲击涨停**
        if c[-1] >= c[-2] and c[-1] >= o[-1] and h[-1]/c[-2]-1 > 0.07:
            vr = volume[-1]/np.mean(volume[:-1])
            results.append((code, name, vr))
        
    if codes is None:
        results = await scan(find_vr, 40, '30m', end, nstocks=-1)
    else:
        results = await scan(find_vr, 40, '30m', end, codes=codes)

    results = sorted(results, key = lambda x: x[2], reverse=True)
    
    
    # add RSI
    final_result = []
    for code, name, vr in results[:100]:
        bars = await get_bars(code, 20, end=day)
        if len(bars) < 20:
            continue
            
        rsi = relative_strength_index(bars["close"])
        prsi = [rsiday.get_proba(code, r) for r in rsi[-2:]]
        
        if prsi[0] is None:
            final_result.append((code, name, vr, round(max(rsi[-2:]),1)))
        else:
            final_result.append((code, name, vr, round(max(prsi[-2:]) * 100, 1)))
         
    return final_result

In [4]:
init_jq()
def check(pool, dec=0):
    """14:30分运行，按日线量比排序"""
    def feature(code, name, bars):
        volume = bars["volume"]
        close = bars["close"]
        open_ = bars["open"]
        
        # 10:00以后下跌量小于0.5
        vdown = 0
        for i in range(1, len(bars)):
            if volume[i] > volume[0] * 0.5 and (close[i] < open_[i] or close[i] < close[i-1]):
                return None
            else:
                vdown = max(vdown, volume[i]/volume[0])
           
        if vdown < 0.5:
            return vdown
                
    
    results = []
    now = arrow.now()
    start = datetime.datetime(now.year, now.month, now.day, 10, 1)
    end = now.datetime
    
    for (code, name, vr, rsi) in pool:
        bars = jq.get_price(code, start_date = start, end_date=end, frequency='30m', panel=False)
            
        vdown = feature(code, name, bars)
        if vdown is None:
            continue
        
        day_bars_end = end
        day_bars_start = tf.day_shift(end, -5)
        
        day_bars = jq.get_price(code, start_date = day_bars_start, end_date = day_bars_end, panel=False)
        #print(day_bars)
        # 计算涨幅
        adv = round(day_bars["close"][-1] / day_bars["close"][-2] - 1,3)
        if adv > 0.05:
            continue
        
        # 计算日线量比
        lb = day_bars["volume"][-1] / np.average(day_bars["volume"][:-1])
            
        results.append((code, name, vr, vdown, rsi, adv, lb))
        
    df = pd.DataFrame(results,columns=["code", "name", "vup", "vdown", "rsi", "adv", "day_lb"])
    return df.sort_values(by = 'vup', ascending=False)

auth success 


In [5]:
async def pooling_3_days(n=20):
    pool = []
    today = arrow.now()
    for tm in tf.get_frames_by_count(today, 3, FrameType.DAY):
        dt = tf.int2date(tm)
        pool.extend(await pooling())
        
    added = set()
    unique_pool = []
    for p in pool:
        if p[0] not in added:
            added.add(p[0])
            unique_pool.append(p)
            
    return sorted(unique_pool, key = lambda x: x[2], reverse=True)[:n]

## 30分钟单日

In [6]:
pool_today = await pooling()
clear_output()
len(pool_today)

90

In [7]:
df = check(pool_today)
df

,code,name,vup,vdown,rsi,adv,day_lb
0,002177.XSHE,御银股份,20.413422,0.487717,97.0,0.027,3.718243
1,603518.XSHG,锦泓集团,16.725758,0.342265,77.0,0.034,4.513750
2,002054.XSHE,德美化工,12.812697,0.334360,99.0,0.035,2.990742
3,002955.XSHE,鸿合科技,9.979736,0.361567,73.8,0.039,2.606626
4,002118.XSHE,紫鑫药业,9.937180,0.290281,99.0,0.034,3.109658
5,300045.XSHE,华力创通,8.591647,0.462491,82.0,0.031,1.780633
6,603138.XSHG,海量数据,8.154691,0.479126,94.0,0.035,1.868991
7,603439.XSHG,贵州三力,7.656188,0.402921,77.2,0.010,1.535921
8,600128.XSHG,弘业股份,6.770404,0.412817,90.0,-0.010,1.431375
9,600992.XSHG,贵绳股份,3.788667,0.270860,93.0,0.047,1.679758


## 历史版

In [20]:
pool = await pooling()
poo

2021-12-20 10:00:00+08:00
progress: 500/4033, results: 191, elapsed: 0, ETA: 0
progress: 1000/4033, results: 354, elapsed: 1, ETA: 3
progress: 1500/4033, results: 527, elapsed: 2, ETA: 3
progress: 2000/4033, results: 668, elapsed: 3, ETA: 3
progress: 2500/4033, results: 808, elapsed: 4, ETA: 2
progress: 3000/4033, results: 999, elapsed: 5, ETA: 1
progress: 3500/4033, results: 1155, elapsed: 6, ETA: 0
progress: 4000/4033, results: 1300, elapsed: 7, ETA: 0


[('300878.XSHE', '维康药业', 44.40360766629087, -0.01343834400177002),
 ('002082.XSHE', '万邦德', 42.32667876588022, 0.0018956661224365234),
 ('300534.XSHE', '陇神戎发', 33.67501581277673, 0.03366994857788086),
 ('300560.XSHE', '中富通', 28.297591099316115, -0.007547199726104736),
 ('603918.XSHG', '金桥信息', 28.23875728862974, 0.01453089714050293),
 ('300519.XSHE', '新光药业', 27.496809138199175, -0.011543035507202148),
 ('603421.XSHG', '鼎信通讯', 27.153703649248683, 0.0506666898727417),
 ('003008.XSHE', '开普检测', 26.204700155092194, 0.11659348011016846),
 ('600605.XSHG', '汇通能源', 25.633416836035355, 0.13003456592559814),
 ('300683.XSHE', '海特生物', 25.13664596273292, -0.0014827251434326172),
 ('002644.XSHE', '佛慈制药', 24.284805735430158, -0.00539088249206543),
 ('002118.XSHE', '紫鑫药业', 21.65303624697881, 0.026119351387023926),
 ('603861.XSHG', '白云电器', 21.309404926728703, 0.1412104368209839),
 ('000965.XSHE', '天保基建', 20.855942091557207, 0.06276142597198486),
 ('300721.XSHE', '怡达股份', 20.844680943865313, -0.000782668590